In [1]:
import glob
import os
from datetime import datetime, timedelta, date
from os import path

from igraph import *

import seaborn as sns
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# Various datasets to visualize

option = 1

if option == 1:
    base_folder = "./osc2020"
#     ext = "csv" # Extension of log file, either log or txt
#     ntot = 20   # Total number of participants
#     vstep = 2   # Time step for visualization
#     ostep = 1   # Time step for output to R
#     infer_new_cases_from_inf = True # Time for "new case" is inferred either from infection or removal time
#     asympt = 2  # Asymptomatic period
#     sympt = 2   # Symptomatic period
#     start_sim = datetime.strptime("2020-03-05 13:30", '%Y-%m-%d %H:%M')
#     stop_sim = datetime.strptime("2020-03-05 14:15", '%Y-%m-%d %H:%M')

In [140]:
# Some config

# Coded outcomes:
# 0 = no infection
# 1 = index case
# 2 = transmission
color_dict = {0: "Lime Green", 1: "White", 2: "Dark Orange"}

data_folder = path.join(base_folder, "data")
output_folder = path.join(base_folder, "output")
if not path.exists(output_folder):
    os.mkdir(output_folder)
    
# viz_delta = timedelta(minutes=vstep)
# out_delta = timedelta(minutes=ostep)  

In [141]:
# Load participants and histories

users = pd.read_csv(path.join(data_folder, "participants.csv")) 
histories = pd.read_csv(path.join(data_folder, "histories.csv"))

p2pToSim = pd.Series(users.sim_id.values,index=users.p2p_id).to_dict()

# dfc = participants[participants["sim_id"] == 5]
# dictc = pd.Series(dfc.id.values,index=dfc.p2p_id).to_dict()

# dfs = participants[participants["sim_id"] == 6]
# dicts = pd.Series(dfs.id.values,index=dfs.p2p_id).to_dict()

# center group
# gr = 5 
# fn = "contacts-center.pdf"

#south group
gr = 6 
fn = "contacts-south.pdf"

# north group
# gr = 7 
# fn = "contacts-north.pdf"

dfn = participants[participants["sim_id"] == gr]
hn = histories[histories["sim_id"] == gr]
cn = hn[hn["type"] == "contact"]

df = dfn
p2pToId = pd.Series(df.id.values, index=df.p2p_id).to_dict()
idTop2p = pd.Series(df.p2p_id.values, index=df.id).to_dict()
        
user_index = {}
idx = 0
for kid in idTop2p:
    user_index[kid] = idx
    idx += 1

        
print(len(df))
print(len(idTop2p))    
print(len(p2pToId))
print(len(user_index))
print(len(infMap))

121
121
121
121
8


In [142]:
# Generate contacts

node0 = cn.user_id.values
node1 = cn.peer_id.values

contacts = {}
for id0, p2p1 in zip(node0, node1):
    n0 = user_index[id0]
    if not p2p1 in p2pToId:
        print("Possible contact with group", p2pToSim[p2p1])
        continue
    n1 = user_index[p2pToId[p2p1]]
    if n1 < n0:
        t = n1
        n1 = n0
        n0 = t
    p01 = (n0, n1)
    if p01 in contacts:
        c = contacts[p01]
    else: 
        c = 0
    contacts[p01] = c + 1
    
print(contacts)

Possible contact with group 5
{(37, 38): 2, (5, 38): 6, (4, 25): 13, (29, 31): 2, (38, 42): 2, (38, 44): 1, (38, 45): 2, (29, 30): 1, (20, 29): 1, (59, 60): 94, (58, 59): 3, (58, 60): 13, (60, 69): 184, (59, 69): 4, (51, 70): 30, (14, 73): 214, (59, 84): 3, (45, 86): 3, (44, 86): 2, (51, 59): 4, (44, 94): 1, (51, 97): 1, (51, 96): 1, (21, 33): 1, (51, 99): 1}


In [143]:
# Generate infections and outcomes

inf = hn[hn["type"] == "infection"]
infMap = pd.Series(inf.inf.values, index=inf.user_id).to_dict()
outcomes = [0] * len(df)

for kid in infMap:
    src = infMap[kid]
    idx = user_index[kid]
    if "CASE0" in src:
        outcomes[idx] = 1
    if "PEER" in src:
        outcomes[idx] = 2

In [144]:
nvert = len(user_index)

g = Graph(directed=False)
g.add_vertices(nvert)

g.vs["outcome"] = outcomes
g.vs["color"] = [color_dict[out] for out in g.vs["outcome"]]

for p in contacts:
    n0 = p[0]
    n1 = p[1]    
    w = contacts[p]
    g.add_edges([(n0, n1)])

layout = g.layout("fr")
p = plot(g, os.path.join(output_folder, fn), 
         layout = layout, vertex_size = 9, edge_arrow_width = 1, edge_arrow_size = 0.5, bbox=(0, 0, 800, 800))